In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm

In [7]:
cities = pd.read_csv("worldcities.csv")
cities

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37977000.0,1392685764
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000.0,1360771077
2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1356872604
3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1356226629
4,Manila,Manila,14.5958,120.9772,Philippines,PH,PHL,Manila,primary,23088000.0,1608618140
...,...,...,...,...,...,...,...,...,...,...,...
26564,Nord,Nord,81.7166,-17.8000,Greenland,GL,GRL,Sermersooq,NaN,10.0,1304217709
26565,Timmiarmiut,Timmiarmiut,62.5333,-42.2167,Greenland,GL,GRL,Kujalleq,NaN,10.0,1304206491
26566,Cheremoshna,Cheremoshna,51.3894,30.0989,Ukraine,UA,UKR,Kyyivs’ka Oblast’,NaN,0.0,1804043438
26567,Ambarchik,Ambarchik,69.6510,162.3336,Russia,RU,RUS,Sakha (Yakutiya),NaN,0.0,1643739159


In [8]:
wiki_base_url = "https://en.wikipedia.org/wiki/"
# for saving the areas of the cities
city_areas = {}

### Crawling the Areas

1. Problem: The box on the right city of each of these city Wikipedia articles is a html table and contains rows and areas. The problem here is, that there are sometimes multiple entries for the area, denoting different area properties. In order to solve this problem, we will simply take the largest of these entries.

In [15]:


# go through every city
for city in tqdm(cities["city"]):
    if not city in city_areas.keys():
        # get the html
        page = requests.get(wiki_base_url + city.replace(" ", "_"))

        # transform to soup object
        soup = BeautifulSoup(page.content, 'html.parser')

        # find the table with all information (right, top of wikipedia article)
        table = soup.find('table', class_='infobox geography vcard')


        # if no table was found, something wrent wong and we insert None and deal with it later
        if not table:
            print("Not Found: ", city)
            city_areas[city] = None
        else:
            # for saving all tale entries of one webpage that containg km²
            temporary_list = []

            # get all rows of the table on the right of the wikipedia article
            rows = table.find_all("tr")

            # go over all rows
            for indx, row in enumerate(rows):
                # get all the cells withing the rows
                cells = row.find_all('td')
                # go through each cell
                for cell in cells:
                    # finding all km² entries and putting them into the temporary list, together with 
                    # the index of the corresponding row
                    # also delete the comma that separates thousand from hundreds
                    if "km2" in cell.text:
                        temporary_list.append((indx, float(re.sub("[^0-9.]", "", re.findall(r"\d{1,6}\.?\d{0,2}", cell.text.replace(",", ""))[0]))))
                        if not float(re.sub("[^0-9.]", "", re.findall(r"\d{1,6}\.?\d{0,2}", cell.text.replace(",", ""))[0])):
                            print("problem", city, ":", cell.text)

            temporary_list_2 = []
            found_indx = False
            for indx, cell_v in temporary_list:

                if not found_indx:
                    found_indx = indx
                    temporary_list_2.append(cell_v)
                else:
                    if indx != found_indx + 1:
                        break
                    else:
                        temporary_list_2.append(cell_v)
                        found_indx += 1
            if not temporary_list_2:
                city_areas[city] = None
                continue
            city_areas[city] = max(temporary_list_2)


  2%|▏         | 484/26569 [00:00<00:21, 1210.03it/s]

Not Found:  Yucheng
Not Found:  Ximeicun
Not Found:  Jianguang
Not Found:  Xushan
Not Found:  Huangshan
Not Found:  Huazhou
problem Shīrāz : 0 km2 (0 sq mi)  0%


  2%|▏         | 511/26569 [00:16<1:51:46,  3.89it/s]

Not Found:  Kawanakajima
Not Found:  Lianjiang


  2%|▏         | 516/26569 [00:18<3:10:21,  2.28it/s]

...

  3%|▎         | 847/26569 [03:10<3:23:41,  2.10it/s]

Not Found:  Tongjin


  3%|▎         | 853/26569 [03:13<1:36:59,  4.42it/s]

Not Found:  Luocheng


ConnectionError: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /wiki/Guw%C4%81h%C4%81ti (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f31ea6a61f0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [5]:
print(city_areas)

{}
